In [6]:
import os
import geopandas as gpd
import certifi
import requests
import pandas as pd
import matplotlib.pyplot as plt
import logging
import time

from urllib.parse import quote
from cdsetool.query import query_features, shape_to_wkt
from cdsetool.credentials import Credentials
from cdsetool.download import download_features
from cdsetool.monitor import StatusMonitor
from datetime import date, timedelta
from shapely.geometry import mapping
from dotenv import load_dotenv
from shapely.wkt import loads
from datetime import datetime
import rasterio
from rasterio.mask import mask

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

env_path = "/Users/luryand/Documents/encode-image/coverage_otimization/code/.env"
load_dotenv(env_path)
logger.info(f"Load credentials")

username = os.getenv("CDSE_USER")
password = os.getenv("CDSE_PASSWORD")
logger.info(f"Success Auth")

shapefile_path = "/Users/luryand/Documents/encode-image/coverage_otimization/code/APA-input/bounding-search.shp"
output_dir = "/volume/luryand"
start_date = date(2024, 1, 1)
end_date = date(2025, 4, 3)
cloud_cover = (0, 100)

os.environ['SSL_CERT_FILE'] = certifi.where()

In [8]:
def load_app_polygons(shapefile_path, priority_column=None):
    try:
        gdf = gpd.read_file(shapefile_path)
        print(f"Shapefile carregado: {len(gdf)} polígonos")
        print(f"CRS original: {gdf.crs}")
    
        if gdf.crs != "EPSG:4326":
            gdf = gdf.to_crs("EPSG:4326")
            print(f"Projeção transformada para WGS 84")
    
        if priority_column and priority_column in gdf.columns:
            gdf['priority'] = gdf[priority_column]
        else:
            gdf['priority'] = 1
            print("Prioridade 1 atribuída a todas as APPs")
        
        gdf['geometry'] = gdf['geometry'].buffer(0)
        invalid_geoms = ~gdf['geometry'].is_valid
        if invalid_geoms.any():
            print(f"Corrigindo {invalid_geoms.sum()} geometrias inválidas")
        
        return gdf
    except Exception as e:
        print(f"Erro ao carregar shapefile: {str(e)}")
        return None

def prepare_footprints(gdf):
    footprints = []
    for idx, row in gdf.iterrows():
        wkt = row['geometry'].wkt
        footprints.append({
            'id': idx,
            'wkt': wkt,
            'priority': row['priority']
        })
    return footprints

In [9]:
class CDSEQuerier:
    def __init__(self, username=None, password=None):
        self.username = username
        self.password = password
        self.token = None
        self.api_url = 'https://catalogue.dataspace.copernicus.eu/resto/api/'
        self.token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
        self.token = self.get_access_token()
        if self.token:
            logger.info("API CDSE inicializada com autenticação baseada em token")
        else:
            logger.error("Falha ao inicializar API CDSE - token não obtido")
    
    def get_access_token(self):
        try:
            payload = {
                "client_id": "cdse-public",
                "username": self.username,
                "password": self.password,
                "grant_type": "password",
            }
            headers = {"Content-Type": "application/x-www-form-urlencoded"}
            response = requests.post(self.token_url, data=payload, headers=headers)
            if response.status_code == 200:
                token = response.json().get("access_token")
                logger.info("Token de acesso obtido com sucesso")
                return token
            else:
                logger.error(f"Falha ao obter token: {response.status_code} - {response.text}")
                return None
        except Exception as e:
            logger.error(f"Erro ao obter token de acesso: {str(e)}")
            return None

    def query_sentinel(self, footprint, start_date, end_date, platform='Sentinel-2', cloud_cover=None, product_type=None):
        if not self.token:
            logger.info("Token não definido. Obtendo novo token...")
            self.token = self.get_access_token()
            if not self.token:
                logger.error("Não foi possível obter um token válido")
                return pd.DataFrame()
        
        collection = "Sentinel1" if platform == "Sentinel-1" else "Sentinel2"
        base_url = f"{self.api_url}collections/{collection}/search.json"
        params = {
            "geometry": footprint,
            "startDate": start_date.strftime("%Y-%m-%d"),
            "completionDate": end_date.strftime("%Y-%m-%d"),
            "maxRecords": 500
        }
        if platform == 'Sentinel-2' and cloud_cover is not None:
            params["cloudCover"] = f"[{cloud_cover[0]},{cloud_cover[1]}]"
        if product_type is not None:
            params["productType"] = product_type
        headers = {"Authorization": f"Bearer {self.token}"}
        try:
            response = requests.get(base_url, params=params, headers=headers)
            if response.status_code != 200:
                logger.error(f"Erro na consulta: HTTP status {response.status_code}")
                return pd.DataFrame()
            data = response.json()
            features = data.get('features', [])
            products = []
            for feature in features:
                properties = feature.get('properties', {})
                product = {
                    'title': properties.get('title'),
                    'id': feature.get('id'),
                    'beginposition': properties.get('startDate'),
                    'endposition': properties.get('completionDate'),
                    'ingestiondate': properties.get('published'),
                    'footprint': feature.get('geometry'),
                }
                if platform == 'Sentinel-2' and 'cloudCover' in properties:
                    product['cloudcoverpercentage'] = properties.get('cloudCover')
                products.append(product)
            df = pd.DataFrame(products)
            for date_col in ['beginposition', 'endposition', 'ingestiondate']:
                if date_col in df.columns:
                    df[date_col] = pd.to_datetime(df[date_col])
            logger.info(f"Encontrados {len(df)} produtos {platform}")
            return df
        except Exception as e:
            logger.error(f"Erro na consulta: {str(e)}")
            return pd.DataFrame()

In [10]:
# aqui
gdf = load_app_polygons(shapefile_path, priority_column="priority")
if gdf is not None:
    footprints = prepare_footprints(gdf)
    print(f"Footprints criados: {len(footprints)}")
else:
    footprints = []
    print("Nenhum footprint foi criado devido a erro no carregamento do shapefile.")

try:
    querier = CDSEQuerier(username, password)
    results_s1 = []
    results_s2 = []
    
    for i, fp in enumerate(footprints):
        print(f"Consultando APP {i+1}/{len(footprints)} (ID: {fp['id']})")
        geom = loads(fp['wkt'])
        geom_simplified = geom.simplify(0.01, preserve_topology=True)
        wkt_simplified = geom_simplified.wkt
        
        # Consulta Sentinel-2
        df_s2 = querier.query_sentinel(
            footprint=wkt_simplified,
            start_date=start_date,
            end_date=end_date,
            platform='Sentinel-2',
            cloud_cover=cloud_cover
        )
        
        if not df_s2.empty:
            # Baixa os produtos Sentinel-2 completos (.SAFE)
            print(f"Iniciando download de {len(df_s2)} produtos Sentinel-2 completos (.SAFE)")
            cdse_credentials = Credentials(username, password)
            downloads = download_features(
                df_s2,
                "/tmp",  # diretório onde os arquivos serão salvos
                {
                    "concurrency": 4,
                    "monitor": StatusMonitor(),
                    "credentials": cdse_credentials,
                }
            )
            for prod_id in downloads:
                print(f"Produto Sentinel-2 completo (.SAFE) {prod_id} baixado com sucesso.")
            
            results_s2.append(df_s2)
        else:
            print("Nenhum produto Sentinel-2 encontrado para este footprint.")
        
        # Consulta Sentinel-1
        df_s1 = querier.query_sentinel(
            footprint=wkt_simplified,
            start_date=start_date,
            end_date=end_date,
            platform='Sentinel-1',
            product_type='GRD'
        )
        if not df_s1.empty:
            # Opcionalmente, você pode adicionar código para baixar os produtos Sentinel-1 aqui
            # similar ao que foi feito para Sentinel-2
            print(f"Encontrados {len(df_s1)} produtos Sentinel-1 (sem download)")
            results_s1.append(df_s1)
        else:
            print("Nenhum produto Sentinel-1 encontrado para este footprint.")
    
    # Consolidação de resultados Sentinel-2
    if results_s2:
        all_s2 = pd.concat(results_s2)
        print(f"Total de produtos Sentinel-2 encontrados: {len(all_s2)}")
    else:
        print("Nenhum produto Sentinel-2 encontrado")
    
    # Consolidação de resultados Sentinel-1
    if results_s1:
        all_s1 = pd.concat(results_s1)
        print(f"Total de produtos Sentinel-1 encontrados: {len(all_s1)}")
    else:
        print("Nenhum produto Sentinel-1 encontrado")

except Exception as e:
    print(f"Erro inesperado: {str(e)}")

In [ ]:
class oi:
    # Carregar os polígonos do shapefile e preparar os footprints
    gdf = load_app_polygons(shapefile_path, priority_column="priority")

    # Preparar os footprints
    if gdf is not None:
        footprints = prepare_footprints(gdf)
        print(f"Footprints criados: {len(footprints)}")
    else:
        footprints = []
        print("Nenhum footprint foi criado devido a erro no carregamento do shapefile.")
        
    class CDSEQuerier:
        def __init__(self, username=None, password=None):
            self.username = username
            self.password = password
            self.token = None
            self.api_url = 'https://catalogue.dataspace.copernicus.eu/resto/api/'
            self.token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
        
            self.token = self.get_access_token()
            if self.token:
                logger.info("API CDSE inicializada com autenticação baseada em token")
            else:
                logger.error("Falha ao inicializar API CDSE - token não obtido")
        
        def get_access_token(self):
            try:
                payload = {
                    "client_id": "cdse-public",
                    "username": self.username,
                    "password": self.password,
                    "grant_type": "password",
                }
                
                headers = {
                    "Content-Type": "application/x-www-form-urlencoded"
                }
                
                response = requests.post(self.token_url, data=payload, headers=headers)
                
                if response.status_code == 200:
                    token = response.json().get("access_token")
                    logger.info("Token de acesso obtido com sucesso")
                    return token
                else:
                    logger.error(f"Falha ao obter token: {response.status_code} - {response.text}")
                    return None
                    
            except Exception as e:
                logger.error(f"Erro ao obter token de acesso: {str(e)}")
                return None

        def query_sentinel(self, footprint, start_date, end_date, 
                        platform='Sentinel-2', cloud_cover=None, product_type=None):
            if not self.token:
                logger.info("Token não definido. Obtendo novo token...")
                self.token = self.get_access_token()
                if not self.token:
                    logger.error("Não foi possível obter um token válido")
                    return pd.DataFrame()
            
            collection = "Sentinel1" if platform == "Sentinel-1" else "Sentinel2"
            base_url = f"{self.api_url}collections/{collection}/search.json"
            
            params = {
                "geometry": footprint,
                "startDate": start_date.strftime("%Y-%m-%d"),
                "completionDate": end_date.strftime("%Y-%m-%d"),
                "maxRecords": 500
            }
            
            if platform == 'Sentinel-2' and cloud_cover is not None:
                params["cloudCover"] = f"[{cloud_cover[0]},{cloud_cover[1]}]"
            
            if product_type is not None:
                params["productType"] = product_type
            
            headers = {"Authorization": f"Bearer {self.token}"}
            
            logger.info(f"Consultando {platform} de {start_date} a {end_date}")
            
            try:
                response = requests.get(base_url, params=params, headers=headers)
                
                if response.status_code != 200:
                    if response.status_code in [401, 403]:
                        logger.info("Erro de autenticação. Renovando token...")
                        self.token = self.get_access_token()
                        if self.token:
                            headers = {"Authorization": f"Bearer {self.token}"}
                            response = requests.get(base_url, params=params, headers=headers)
                            if response.status_code != 200:
                                logger.error(f"Erro na consulta mesmo após renovação do token: {response.status_code}")
                                return pd.DataFrame()
                        else:
                            return pd.DataFrame()
                    else:
                        logger.error(f"Erro na consulta: HTTP status {response.status_code}")
                        return pd.DataFrame()
                
                data = response.json()
                features = data.get('features', [])
                
                if not features:
                    logger.warning(f"Nenhum produto {platform} encontrado")
                    return pd.DataFrame()
                
                products = []
                for feature in features:
                    properties = feature.get('properties', {})
                    product = {
                        'title': properties.get('title'),
                        'id': feature.get('id'),
                        'beginposition': properties.get('startDate'),
                        'endposition': properties.get('completionDate'),
                        'ingestiondate': properties.get('published'),
                        'footprint': feature.get('geometry'),
                    }
                    
                    if platform == 'Sentinel-2' and 'cloudCover' in properties:
                        product['cloudcoverpercentage'] = properties.get('cloudCover')
                    
                    products.append(product)
                
                df = pd.DataFrame(products)
                
                for date_col in ['beginposition', 'endposition', 'ingestiondate']:
                    if date_col in df.columns:
                        df[date_col] = pd.to_datetime(df[date_col])
                
                logger.info(f"Encontrados {len(df)} produtos {platform}")
                return df
                
            except Exception as e:
                logger.error(f"Erro na consulta: {str(e)}")
                return pd.DataFrame()

    try:
        querier = CDSEQuerier(username, password)
        
        results_s1 = []
        results_s2 = []

        for i, fp in enumerate(footprints):
            print(f"Consultando APP {i+1}/{len(footprints)} (ID: {fp['id']})")
            
            geom = loads(fp['wkt'])
            geom_simplified = geom.simplify(0.01, preserve_topology=True)
            wkt_simplified = geom_simplified.wkt

            df_s2 = querier.query_sentinel(
                footprint=wkt_simplified,
                start_date=start_date,
                end_date=end_date,
                platform='Sentinel-2',
                cloud_cover=cloud_cover
            )
            if not df_s2.empty:
                df_s2['app_id'] = fp['id']
                df_s2['priority'] = fp['priority']
                results_s2.append(df_s2)
            
            df_s1 = querier.query_sentinel(
                footprint=wkt_simplified,
                start_date=start_date,
                end_date=end_date,
                platform='Sentinel-1',
                product_type='GRD'
            )
            if not df_s1.empty:
                df_s1['app_id'] = fp['id']
                df_s1['priority'] = fp['priority']
                results_s1.append(df_s1)

        if results_s2:
            all_s2 = pd.concat(results_s2)
            print(f"Total de produtos Sentinel-2 encontrados: {len(all_s2)}")
        else:
            all_s2 = pd.DataFrame()
            print("Nenhum produto Sentinel-2 encontrado")
        
        if results_s1:
            all_s1 = pd.concat(results_s1)
            print(f"Total de produtos Sentinel-1 encontrados: {len(all_s1)}")
        else:
            all_s1 = pd.DataFrame()
            print("Nenhum produto Sentinel-1 encontrado")
            
    except ValueError as e:
        print(f"Erro: {str(e)}")
    except Exception as e:
        print(f"Erro inesperado: {str(e)}")

In [ ]:
import os
import certifi
import requests
import pandas as pd
import logging
import geopandas as gpd
from datetime import date
from dotenv import load_dotenv
from shapely.wkt import loads
from cdsetool.credentials import Credentials
from cdsetool.download import download_features
from cdsetool.monitor import StatusMonitor

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Load credentials
env_path = "/Users/luryand/Documents/encode-image/coverage_otimization/code/.env"
load_dotenv(env_path)
username = os.getenv("CDSE_USER")
password = os.getenv("CDSE_PASSWORD")
logger.info("Credentials loaded")

# Config
os.environ['SSL_CERT_FILE'] = certifi.where()
shapefile_path = "/Users/luryand/Documents/encode-image/coverage_otimization/code/APA-input/bounding-search.shp"
download_dir = os.path.expanduser("~/Downloads/sentinel_data")  # More visible location
os.makedirs(download_dir, exist_ok=True)
start_date = date(2024, 1, 1)
end_date = date(2024, 3, 1)  # Shorter time range
cloud_cover = (0, 30)  # Lower cloud cover for better quality

# Function to get access token
def get_access_token(username, password):
    token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
    payload = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    
    try:
        response = requests.post(token_url, data=payload, headers=headers)
        if response.status_code == 200:
            return response.json().get("access_token")
        else:
            logger.error(f"Token error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        logger.error(f"Token request failed: {e}")
        return None

# Function to query Sentinel images
def query_sentinel(footprint, start_date, end_date, platform='Sentinel-2', cloud_cover=None):
    token = get_access_token(username, password)
    if not token:
        return pd.DataFrame()
    
    collection = "Sentinel1" if platform == "Sentinel-1" else "Sentinel2"
    api_url = 'https://catalogue.dataspace.copernicus.eu/resto/api/'
    base_url = f"{api_url}collections/{collection}/search.json"
    
    params = {
        "geometry": footprint,
        "startDate": start_date.strftime("%Y-%m-%d"),
        "completionDate": end_date.strftime("%Y-%m-%d"),
        "maxRecords": 10  # Limit results
    }
    
    if platform == 'Sentinel-2' and cloud_cover is not None:
        params["cloudCover"] = f"[{cloud_cover[0]},{cloud_cover[1]}]"
        
    headers = {"Authorization": f"Bearer {token}"}
    
    try:
        logger.info(f"Querying {platform} from {start_date} to {end_date}")
        response = requests.get(base_url, params=params, headers=headers)
        
        if response.status_code != 200:
            logger.error(f"Query error: {response.status_code}")
            return pd.DataFrame()
            
        data = response.json()
        features = data.get('features', [])
        logger.info(f"Found {len(features)} {platform} products")
        
        products = []
        for feature in features:
            properties = feature.get('properties', {})
            product = {
                'title': properties.get('title'),
                'id': feature.get('id'),
                'beginposition': properties.get('startDate'),
                'cloudcoverpercentage': properties.get('cloudCover') if platform == 'Sentinel-2' else None
            }
            products.append(product)
            
        return pd.DataFrame(products)
    except Exception as e:
        logger.error(f"Query failed: {e}")
        return pd.DataFrame()

# Main execution
try:
    # Load shapefile
    gdf = gpd.read_file(shapefile_path)
    logger.info(f"Loaded {len(gdf)} polygons from shapefile")
    
    if gdf.crs != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")
    
    # Get first polygon for simplicity
    if len(gdf) > 0:
        geom = gdf.iloc[0].geometry
        wkt = geom.wkt
        logger.info("Using first polygon for query")
        
        # Query for Sentinel-2 images
        df_s2 = query_sentinel(wkt, start_date, end_date, 'Sentinel-2', cloud_cover)
        
        if not df_s2.empty:
            # Show preview of what will be downloaded
            logger.info(f"Found {len(df_s2)} Sentinel-2 images to download:")
            print(df_s2[['title', 'beginposition', 'cloudcoverpercentage']].head())
            
            # Confirm download
            confirm = input(f"\nDownload {len(df_s2)} images to {download_dir}? (y/n): ")
            
            if confirm.lower() == 'y':
                # Download images
                logger.info(f"Downloading {len(df_s2)} images to {download_dir}")
                cdse_credentials = Credentials(username, password)
                
                # Create list of IDs for download
                features_to_download = df_s2['id'].tolist()
                
                downloads = download_features(
                    features_to_download,
                    download_dir,
                    {
                        "concurrency": 2,  # Limit concurrent downloads
                        "monitor": StatusMonitor(),
                        "credentials": cdse_credentials
                    }
                )
                
                logger.info(f"Download complete. {len(downloads)} images downloaded to {download_dir}")
            else:
                logger.info("Download cancelled by user")
        else:
            logger.info("No Sentinel-2 images found matching criteria")
    else:
        logger.error("No polygons found in shapefile")
except Exception as e:
    logger.error(f"Error: {e}")

In [1]:
import os
import logging
from datetime import date
from dotenv import load_dotenv
import certifi
import geopandas as gpd

from cdsetool.query import query_features, shape_to_wkt
from cdsetool.credentials import Credentials
from cdsetool.download import download_features

# Configuração do logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Carrega variáveis de ambiente e credenciais
env_path = "/Users/luryand/Documents/encode-image/coverage_otimization/code/.env"
load_dotenv(env_path)
logger.info("Credenciais carregadas")

username = os.getenv("CDSE_USER")
password = os.getenv("CDSE_PASSWORD")

# Parâmetros de consulta e diretórios
shapefile_path = "/Users/luryand/Documents/encode-image/coverage_otimization/code/APA-input/bounding-search.shp"
output_dir = "/volume/luryand"
start_date = date(2025, 1, 1)
end_date = date(2025, 4, 3)
cloud_cover = (0, 100)

os.environ['SSL_CERT_FILE'] = certifi.where()

# Leitura do shapefile e conversão para WKT
gdf = gpd.read_file(shapefile_path)
geometry_wkt = shape_to_wkt(gdf.unary_union)

# Consulta dos produtos
logger.info("Consultando produtos...")
produtos = query_features(
    credentials=Credentials(username=username, password=password),
    geometry=geometry_wkt,
    start_date=start_date,
    end_date=end_date,
    cloud_cover=cloud_cover
)
logger.info(f"Total de produtos encontrados: {len(produtos)}")

# Download dos produtos encontrados
if produtos:
    logger.info("Iniciando download dos produtos...")
    download_features(
        features=produtos,
        output_directory=output_dir,
        credentials=Credentials(username=username, password=password)
    )
    logger.info("Download concluído.")
else:
    logger.info("Nenhum produto encontrado para o período especificado.")
